In [22]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

import cv2

## Build and Train Model

In [23]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [24]:
data = DataLoader.from_folder(r"./datasets/tensorflow/photos/")
train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 2917, num_label: 2, labels: Moving, still.


INFO:tensorflow:Load image with size: 2917, num_label: 2, labels: Moving, still.


In [25]:
model = image_classifier.create(train_data)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(test_data)

10/10 [==============================] - 1s 45ms/step - loss: 0.2601 - accuracy: 0.9692


In [ ]:
model.export(export_dir='.')

2023-05-26 11:42:43.804419: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpml5_ih03/assets


INFO:tensorflow:Assets written to: /tmp/tmpml5_ih03/assets
2023-05-26 11:42:46.840100: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-26 11:42:46.840210: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-05-26 11:42:46.840264: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-26 11:42:46.840616: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-26 11:42:46.840728: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-26 11:42:46.840827: I t

INFO:tensorflow:Label file is inside the TFLite model with metadata.


fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 3
INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpjhnwg859/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmpjhnwg859/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


## Predict

In [ ]:
def resize_video(input_path, output_path, width, height):
  # Open the video file
  video = cv2.VideoCapture(input_path)

  # Get the original video's width and height
  original_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  original_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  
  # Create a VideoWriter object to save the resized video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for the output video
  fps = video.get(cv2.CAP_PROP_FPS)
  writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
  
  while True:
    # Read a frame from the original video
    ret, frame = video.read()
    if not ret: break
    # Resize the frame to the desired width and height
    resized_frame = cv2.resize(frame, (width, height))
    # Write the resized frame to the output video file
    writer.write(resized_frame)

# Release the video capture and writer objects
  video.release()
  writer.release()

In [46]:
# Resize vid
video_path = "./datasets/vid9.mp4"
output_path = "resized_vid9.mp4"
target_width = 224
target_height = 224

# resize_video(video_path, output_path, target_width, target_height)

# Load resized vid
cap = cv2.VideoCapture(output_path)
# frame_rate = 30
# cap.set(cv2.CAP_PROP_FPS, frame_rate) DOESN'T WORK
print(cap.get(cv2.CAP_PROP_FPS))

30.0


In [47]:
# Initialize list to store the frame classifications
frame_classifications = []

# Loop through the frames of the video (need to change to 30 fps)
while True:
    ret, frame = cap.read() 
    #just need to figure out if this is 30 fps

    if not ret: # Break the loop if the video has ended
        break

    ''' Formulate Input Data (frame_rgb) '''
    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Make frame input data and ensure its type matches the model
    frame_rgb = np.expand_dims(frame_rgb, axis=0)

    ''' Classify the Frame '''
    interpreter = tf.lite.Interpreter(model_path="model.tflite")
    interpreter.allocate_tensors()

    # get_output_details() and get_input_details() return list of dictionaries of tensor details
    # keys: name, index, shape, shape_signature, dtype, quantization, ...
    # len(input) = len(output) = 1, so access the first element
    output = interpreter.get_output_details()
    input = interpreter.get_input_details()
    output_index = output[0]['index']
    input_index = input[0]['index']

    # set input -> invoke -> access output
    interpreter.set_tensor(input_index, frame_rgb)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_index)
    # If the output_data shape is (batch_size, num_classes), select the first frame
    output_data = output_data[0]

    # Convert each entry into probability
    output_probs = tf.nn.softmax(output_data.astype(float))

    # Find the index of the highest probability
    predicted_index = np.argmax(output_data)

    # Assuming you have a list of class labels corresponding to the model's output classes
    class_labels = ["Moving", "Still"]

    # Get the predicted class label
    predicted_class = class_labels[predicted_index]

    # Print the predicted class label
    # print("Predicted Class:", predicted_class)
    frame_classifications.append((predicted_class, max(output_probs.numpy())))
    '''
    print("output: ")
    print(output_data)

    # Display the frame
    cv2.imshow("Video", frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    '''

In [48]:
''' SHOW CLASSIFICATION RESULTS '''
# Convert the frame classifications to a numpy array
#frame_classifications = np.array(predicted_class)
# Print the frame classifications
print("Frame Classifications:", frame_classifications)

Frame Classifications: [('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Still', 1.0), ('Moving', 0.9820137900379085), ('Moving', 0.9820137900379085), ('Moving', 0.9999999999999873), ('Moving', 1.0), ('Moving', 1.0), ('Moving', 1.0), ('Moving', 1.0), ('Moving', 1.0), ('Moving', 1.0), ('Moving', 0.5), ('Moving', 0.9999999997210531), ('Moving', 0.9999999999993086), ('Still',

In [ ]:
# Release the video capture and close the OpenCV windows (Do we need this?) yes, i think thats why i took up all the resources
cap.release()
cv2.destroyAllWindows()